In [ ]:
import notebook_import_fix
from sklearn.linear_model import SGDClassifier
import numpy as np
from mozfldp.simulation_util import client_update
import warnings

# hide the warning message temporarily
warnings.simplefilter("ignore")

# auto-reload the modules everytime a cell is run
%load_ext autoreload
%autoreload 2

## Client Update Example

In [ ]:
# this data will be provided by the server
features = [[1, 4, 3], [0, 2, 2], [1, 4, 0], [0, 5, 3], [1, 2, 1], [0, 2, 9]]
labels = [1, 0, 1, 0, 1, 0]
all_classes = [0, 1]

coefs = np.array([29., 0., 0.]) # should be of size num_classes * num_features
intercepts = np.array([-9.])
weights = [coefs, intercepts]

epochs = 3
batch_size = 3

new_weights = client_update(
    weights,
    epochs,
    batch_size,
    features,
    labels,
    all_classes
)
print(new_weights)

## Server Update Example

In [ ]:
import numpy as np
from mozfldp.simulation_util import server_update

init_weights = [np.array([0., 0., 0.]), np.array([0.])]
client_fraction = 0.5
num_rounds = 10
epoch = 10
batch_size = 25
display_weight_per_round = True

num_client = 100
samples_per_client = 100
num_features = 3
features = np.random.randint(10, size=(num_client, samples_per_client, num_features))
labels = np.random.randint(2, size=(num_client, samples_per_client))

new_clf = server_update(
    init_weights,
    client_fraction,
    num_rounds,
    features,
    labels,
    epoch,
    batch_size,
    display_weight_per_round
)

## Simulation Runner

In [ ]:
from sklearn.model_selection import ParameterGrid, train_test_split
from mozfldp.simulation_util import server_update
import numpy as np
import mozfldp.random_data_gen as rdata_gen
import pandas as pd
from numpy import zeros, newaxis
from sklearn.utils import shuffle

print("Started Training")

# Load the data
NUM_SAMPLES = 70000
NUM_LABELS = 10
NUM_FEATURES = 784
NUM_CLIENTS = 100
g_prms = rdata_gen.InputGenParams(NUM_SAMPLES, NUM_LABELS, NUM_FEATURES, NUM_CLIENTS)
df = pd.read_csv("../datasets/mnist.csv")

sim_labels, sim_features = rdata_gen.transform_data_for_simulator_format(df, g_prms)
features = np.array(sim_features)
labels = np.array(sim_labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.4, random_state=0)

init_weights = np.zeros((NUM_LABELS, NUM_FEATURES), dtype=np.float64, order="C")
init_intercept = np.zeros(NUM_LABELS, dtype=np.float64, order="C")

# Find all the permutations of the parameters
param_grid = {"client_fraction": [0.5],
              "epoch": [10],
              "batch_size": [40, 80], # TODO: need to implement an infinite batch size
              "init_weight": [[init_weights, init_intercept]],
              "num_rounds": [2]}

# run training/testing over all parameter combinations to get the best combination
for params in ParameterGrid(param_grid):
    print("Training...")
    print("Params: ", params)
    classifier = server_update(params["init_weight"], params["client_fraction"], params["num_rounds"], X_train, y_train, params["epoch"], params["batch_size"], False)
    weights = [classifier.coef_, classifier.intercept_]

    # need to remove the client dimension from our data for testing 
    # ex: [[[1, 1], [2, 2]], [[3, 3], [4, 4]]] needs to become [[1, 1], [2, 2], [3, 3], [4, 4]] for features 
    # and [[1, 2], [3, 4]] needs to become [1, 2, 3, 4] for labels 
    reshaped_X_test = np.reshape(X_test, (X_test.shape[0] * X_test.shape[1], X_test.shape[2]))
    reshaped_y_test = np.reshape(y_test, y_test.size)
    
    score = classifier.score(reshaped_X_test, reshaped_y_test)

    print('Weights: {}\nScore: {:f}\n\n'.format(weights, score))